# 입문자를 위한, 파이썬/R 데이터 분석   

## 활용실습(1): 공공데이터 분석 및 지도 시각화 연습

 - 우리나라 인구 지역별 분석 : [KOSIS 국가통계포탈](http://kosis.kr/index/index.do) 
 
 
 - [시군구/성/연령(5세)별 주민등록연앙인구](https://kosis.kr/statHtml/statHtml.do?orgId=101&tblId=DT_1B040M5&vw_cd=MT_ZTITLE&list_id=A_7&seqNo=&lang_mode=ko&language=kor&obj_var_id=&itm_id=&conn_path=MT_ZTITLE) 자료 활용
 
  
 - 시나리오:
 
     1) 목적/목표 :
 
     2) 데이터 확보 :
 
     3) 데이터 정비(Data Cleaning) :
 
     4) 인구 지역별 데이터 정리 :
 
     5) 시각화 작업 :
 
     6) 분석결과 의견 :

### 1) 목적/목표 : 
- 시군구/성/연령(5세)별 주민등록연앙인구 데이터 분석

In [ ]:
import pandas as pd
import numpy as np

### 2) 데이터 확보 :

In [ ]:
# Data 로딩 (xlsx 파일)
# 오류발생시 : !pip install --upgrade openpyxl
population = pd.read_excel('data/population_raw_data.xlsx', header=1)

In [ ]:
population

In [ ]:
population.info()

In [ ]:
population.isnull().sum(0)

### 3) 데이터 정비(Data Cleaning) :

In [ ]:
#population = population.fillna(method='pad')#, inplace=True)  

In [ ]:
population.fillna(method='pad', inplace=True)        # ffill: 결측치NaN 앞의 값으로 채우기, inplace: 업데이트까지 수행 옵션  
                                                     # 번대 : bfill or backfill
population

In [ ]:
population.columns

In [ ]:
# 컬럼명 조정 : 1, 2, 3번째 컬럼 ('항목' 컬럼 -> '구분' 으로 변경)
population.rename(columns = {'행정구역(동읍면)별(1)':'광역시도', '행정구역(동읍면)별(2)':'시도','계':'인구수','항목':'구분'}, inplace=True)

In [ ]:
population.head()

In [ ]:
# 데이터 정비 :
population[population['시도'] != '소계']

In [ ]:
# '총인구수(명) -> '합계'
# '남자인구수(명) -> '남자'
# '여자인구수(명) -> '여자'
population.loc[population['구분'] == '총인구수 (명)', '구분'] = '합계'
population.loc[population['구분'] == '남자인구수 (명)', '구분'] = '남자'
population.loc[population['구분'] == '여자인구수 (명)', '구분'] = '여자'

In [ ]:
population.head(10)

In [ ]:
population[population['시도'] != '소계'].head(10)

### 4) 인구 지역별 데이터 정리 :

In [ ]:
# 연령별 인구수 Group 분류작업 : 20~39세 (20-30대) vs. 65세 이상 

temp1 = '20 - 24세	25 - 29세	30 - 34세	35 - 39세	65 - 69세	70 - 74세	75 - 79세	80 - 84세	85 - 89세	90 - 94세	95 - 99세	100+'
temp1

In [ ]:
temp2 = '20 - 24세	25 - 29세	30 - 34세	35 - 39세	65 - 69세	70 - 74세	75 - 79세	80 - 84세	85 - 89세	90 - 94세	95 - 99세	100+'
temp2.split('\t')

In [ ]:
temp2

In [ ]:
population['20-39세'] = population['20 - 24세'] + population['25 - 29세'] + \
                        population['30 - 34세'] + population['35 - 39세']
    
population['65세이상'] = population['65 - 69세'] + population['70 - 74세'] + \
                        population['75 - 79세'] + population['80 - 84세'] + population['85 - 89세'] + \
                        population['90 - 94세'] + population['95 - 99세'] + population['100+']

In [ ]:
population.head(10)
population[population['시도'] != '소계'].head(10)

In [ ]:
pop = pd.pivot_table(population, index=['광역시도','시도'], columns='구분', values=['인구수','20-39세','65세이상'])
pop

### 참고 : [인구소멸 위험지수](https://ko.wikipedia.org/wiki/%EC%9D%B8%EA%B5%AC%EC%86%8C%EB%A9%B8_%EC%9C%84%ED%97%98%EC%A7%80%EC%88%98)


In [ ]:
pop['20-39세','여자']

In [ ]:
# 인구 소멸비율 컬럼 추가 :
# 참고 : 인구소멸비율 계산식 = 65세이상 고령인구대비 20∼39세 여성인구가 차지하는 비율로 측정

pop['소멸비율'] = pop['20-39세','여자'] / pop['65세이상','합계']

In [ ]:
pop.head()

In [ ]:
pop['소멸비율'] < 0.5

In [ ]:
# 인구 소멸비율 : 0.5 이하로 가정
pop['소멸위기지역'] = pop['소멸비율'] < 0.5     # True: 소멸위기지역,  False: 소멸위기지역 아님
pop.head()

In [ ]:
# 시각화 위한 추가 작업 : Index 그룹 없애기
pop.reset_index()

In [ ]:
pop.reset_index(inplace=True)

pop.head()

#pop = pop.drop('index', axis=1)

In [ ]:
# 시각화 위한 추가 작업 : pivot 첫행 정리 :
pop.columns

In [ ]:
pop.columns.get_level_values(1)

In [ ]:
pop.columns.get_level_values(0)[2]

In [ ]:
pop.columns.get_level_values(1)[2]
#'A' + 'B'

In [ ]:
pop.columns.get_level_values(0)[2] + pop.columns.get_level_values(1)[2]

In [ ]:
top_col = []

for n in range(0, len(pop.columns.get_level_values(0))):
# len(pop.columns.get_level_values(0))    
    print(n, pop.columns.get_level_values(0)[n] + pop.columns.get_level_values(1)[n] )
    top_col.append(pop.columns.get_level_values(0)[n] + pop.columns.get_level_values(1)[n])

In [ ]:
top_col

In [ ]:
pop.columns

In [ ]:
pop.columns = top_col

In [ ]:
pop.info()

In [ ]:
pop.head()

### 5) 시각화 작업 : 

In [ ]:
# 시각화 Folium 으로 지도에 표시위한 추가작업
pop.head(3)

In [ ]:
pop['시도'].unique()

In [ ]:
pop['시도'].str[0:2]

In [ ]:
pop['id'] = pop['시도'].str[0:2]

In [ ]:
pop.head(3)

In [ ]:
pop_map = pop.set_index('id')

In [ ]:
pop_map.head(3)

##### 지역별 인구수합계 시각화

In [ ]:
#가급적 아나콘다 프롬프트에서
#!pip install folium

In [ ]:
# 시각화 지도작업 :
import folium 
import json

In [ ]:
geo_path = 'data/05. skorea_municipalities_geo_simple.json'
geo_data = json.load(open('data/05. skorea_municipalities_geo_simple.json', encoding='utf-8'))

map = folium.Map(location=[36,127], zoom_start=7)

In [ ]:
folium.Choropleth(geo_data=geo_data, 
               data=pop_map['인구수합계'],
               columns=[pop_map.index, pop_map['인구수합계']],
               key_on='feature.id',
               fill_color='YlGnBu').add_to(map)

In [ ]:
geo_data

In [ ]:
map

In [ ]:
map = folium.Map(location=[36.2002, 127.054], zoom_start=7)
folium.Choropleth(geo_data=geo_data, 
               data=pop_map['소멸위기지역'],
               columns=[pop_map.index, pop_map['소멸위기지역']],
               key_on='feature.id',
               fill_color='PuRd').add_to(map)
map

### 6) 분석결과 의견 :

- 수강생 토론

In [ ]:
pd.set_option('display.max_rows', None)  # or 1000

In [ ]:
pop_tmp = pop[pop['소멸위기지역']==True]
pop_tmp

In [ ]:
pop_tmp[pop_tmp['시도']=='고성군']

In [ ]:
len(pop[pop['소멸위기지역']==True])

In [ ]:
len(pop[pop['소멸위기지역']==True])/len(pop)

### [실습과제 : 팀별]

##### 1. 최신 데이터로 업데이트 및 지도 id 텍스트 전처리의 정교화  
   
   - 고성군은 강원도/경상남도 2곳에 존재 
   
   (참고) https://gist.github.com/HyeongWookKim/9ae79f06de0c2e8dd4f66d58db3f8087
   
##### 2. 금융기관 영업점 위치 데이터와 결합한 추가 분석   (추후)
   
   - finance_brach 2021.xlsx 엑셀 파일의 주소 데이터를 광역시/시도로 구분하여 집계     
   
   - 인구 데이터와 Merge 하여 분석 및 시각화
   
##### 3. 스타벅스 영업점 위치 데이터와 결합한 추가 분석  (추후)
   
   - 스타벅스 영업점 주소 데이터를 광역시/시도로 구분하여 집계     
   
   - 인구 데이터와 Merge 하여 분석 및 시각화
    
      (참고) https://mizykk.tistory.com/82